In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [18]:
df = pd.read_csv("../Data/foot.csv")

In [19]:
def map_result(result):
    if result == 'A':
        return 0
    else :
        return 1



df_new = df[df["FTR"] != "D"]
df_new["result"] = df_new['FTR'].apply(map_result)


C:\Users\User\AppData\Local\Temp\ipykernel_19884\1565405666.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["result"] = df_new['FTR'].apply(map_result)


In [20]:
## Creating a list of betting odds column fields.
bettingStats = ["B365H","B365D","B365A","BWH","BWD","BWA","GBH","GBD","GBA","IWH","IWD","IWA","LBH","LBD","LBA","SBH","SBD","SBA","WHH","WHD","WHA","SJH","SJD","SJA","VCH","VCD","VCA","Bb1X2","BbMxH","BbAvH","BbMxD","BbAvD","BbMxA","BbAvA","BbOU","BbMx>2.5","BbAv>2.5","BbMx<2.5","BbAv<2.5","BbAH","BbAHh","BbMxAHH","BbAvAHH","BbMxAHA","BbAvAHA"]
bettingStatsEx = ["BSH", "BSD", "BSA", "PSA", "PSH", "PSD", "PSCA", "PSCD", "PSCH"]

## Creating a list of general unnecesaary column fields.
genDropInfo = ["Div", "Date", "HomeTeam", "AwayTeam", "Referee"]

## Creating a list of unnecessary pre-existing feature columns.
preDropFeatures = ["FTHG", "FTAG", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HTR", "HS", "AS", "HF", "AF", "HY", "AY", "HR", "AR", "HC", "AC", "MHTGD", "MATGD"]

old_result = ["FTR"]


In [21]:
# df_new = df_new.drop(["AwayTeam","HomeTeam",'FTR','Date',"Div",
#                   "HTR","Referee","Season"], axis=1)

# # drop value with lot of None value
# df_new = df_new.drop(["BSA","BSD","BSH",
#                       "PSA","PSCA",
#                       "PSCD","PSCH","PSD","PSH",
#                       "SBA","SBD","SBH","SJA","SJD","SJH"], axis=1)

## Dropping the above coloumns .
df_new.drop(bettingStats + bettingStatsEx + genDropInfo + preDropFeatures + old_result, axis = 1, inplace = True)
# df_new = df_new.drop(["AR ","AS", "AST", "AY"])


df_new = df_new.dropna(subset=df_new.columns)
print(len(df_new))

3108


C:\Users\User\AppData\Local\Temp\ipykernel_19884\3245000978.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.drop(bettingStats + bettingStatsEx + genDropInfo + preDropFeatures + old_result, axis = 1, inplace = True)


In [22]:
## For our "Target Variable" , we need the match outcomes which is the field "FTR" (Full Time Result) .
## So , we will create a slice of this coloumn as well as "Season" (for indexing purposes) .
Y = df_new[['result','Season']]

## Now , we will choose the first 11 seasons as our Training Set and the final 2 as Testing Set .
y_train = Y[ (Y['Season'] == '2005-2006') | (Y['Season'] == '2006-2007') | (Y['Season'] == '2007-2008') | (Y['Season'] == '2008-2009') | (Y['Season'] == '2009-2010') | (Y['Season'] == '2010-2011') | (Y['Season'] == '2011-2012') | (Y['Season'] == '2012-2013') | (Y['Season'] == '2013-2014') | (Y['Season'] == '2014-2015') | (Y['Season'] == '2015-2016')]         
y_test = Y[ (Y['Season'] == '2016-2017') | (Y['Season'] == '2017-2018')]

## Now , we dont need the "Season" coloumn so drop it .
y_train.drop(['Season'], axis = 1, inplace = True)
y_test.drop(['Season'], axis = 1, inplace = True)

C:\Users\User\AppData\Local\Temp\ipykernel_19884\156534692.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.drop(['Season'], axis = 1, inplace = True)
C:\Users\User\AppData\Local\Temp\ipykernel_19884\156534692.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.drop(['Season'], axis = 1, inplace = True)


In [23]:
## Now, for creating the feature set, we first need to remove the target variable from the DataFrame .
X = df_new
X.drop(['result'], axis = 1, inplace = True)

## Now , we will choose the first 11 seasons as our Training Set and the final 2 as Testing Set .
X_train = X[ (X['Season'] == '2005-2006') | (X['Season'] == '2006-2007') | (X['Season'] == '2007-2008') | (X['Season'] == '2008-2009') | (X['Season'] == '2009-2010') | (X['Season'] == '2010-2011') | (X['Season'] == '2011-2012') | (X['Season'] == '2012-2013') | (X['Season'] == '2013-2014') | (X['Season'] == '2014-2015') | (X['Season'] == '2015-2016')] 
X_test = X[ (X['Season'] == '2016-2017') | (X['Season'] == '2017-2018')]

## Now , we dont need the "Season" coloumn so drop it .
X_train.drop(['Season'], axis = 1, inplace = True)
X_test.drop(['Season'], axis = 1, inplace = True)


C:\Users\User\AppData\Local\Temp\ipykernel_19884\2217996624.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['Season'], axis = 1, inplace = True)
C:\Users\User\AppData\Local\Temp\ipykernel_19884\2217996624.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(['Season'], axis = 1, inplace = True)


In [24]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)


scaled_X_train = pd.DataFrame(data=scaled_X_train, columns=X_train.columns)
scaled_X_test = pd.DataFrame(data=scaled_X_test, columns=X_test.columns)

# decision trees


In [25]:
# # Define the decision tree classifier

# dt_classifier = DecisionTreeClassifier()

# # Define the grid search parameters
# param_grid = {
#     'criterion': ['gini', 'entropy'],
#     'splitter': ['best', 'random'],
#     'max_depth': [None, 5, 10, 15, 20],
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf': [1, 2, 4, 8],
#     'max_features': ['auto', 'sqrt', 'log2', None],
#     'class_weight': [None, 'balanced'],
#     'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3]
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params =  {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'splitter': 'random'}


In [26]:
# Train the model with the best parameters
best_dt_classifier = DecisionTreeClassifier(**best_params)
best_dt_classifier.fit(scaled_X_train, y_train)


y_pred = best_dt_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'splitter': 'random'}
Test Accuracy: 0.6639004149377593
Test f1 score: 0.7428571428571429
Test roc auc : 0.6321040372670808


# k nearest neighbors


In [27]:
# Define the K-Nearest Neighbors classifier

# knn_classifier = KNeighborsClassifier()

# # Define the grid search parameters
# param_grid = {
#     'n_neighbors': [3, 5, 7, 9, 11],  # Number of neighbors to consider
#     'weights': ['uniform', 'distance'],  # Weight function used in prediction
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
#     'leaf_size': [10, 20, 30, 40],  # Leaf size passed to BallTree or KDTree
#     # 'p': [1, 2, 3],  # Power parameter for the Minkowski metric
#     # 'metric': ['euclidean', 'manhattan', 'chebyshev']  # Distance metric to use for the tree
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params = {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 11, 'weights': 'uniform'}

In [28]:
# Train the model with the best parameters
best_knn_classifier = KNeighborsClassifier(**best_params)
best_knn_classifier.fit(scaled_X_train, y_train)



y_pred = best_knn_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 11, 'weights': 'uniform'}
Test Accuracy: 0.7385892116182573
Test f1 score: 0.8073394495412844
Test roc auc : 0.7197381842910574


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


# Naive Bayes 


In [29]:
# # Define the Gaussian Naive Bayes classifier

# nb_classifier = GaussianNB()

# # Define the grid search parameters
# param_grid = {
#     'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5],
#     'priors': [None, [0.5, 0.5], [0.3, 0.7], [0.7, 0.3]]  # Prior probabilities for each class
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params = {'priors': None, 'var_smoothing': 1e-09}

In [30]:
# Train the model with the best parameters
best_nb_classifier = GaussianNB(**best_params)
best_nb_classifier.fit(scaled_X_train, y_train)

y_pred = best_nb_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'priors': None, 'var_smoothing': 1e-09}
Test Accuracy: 0.7261410788381742
Test f1 score: 0.7716262975778546
Test roc auc : 0.7124912648497554


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# support vector machines

In [31]:
# # Define the Support Vector Machine classifier

# svm_classifier = SVC()

# # Define the grid search parameters
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#     # 'degree': [2, 3, 4],  # Only for polynomial kernel
#     # 'gamma': ['scale', 'auto', 0.1, 1, 10]  # 'scale' and 'auto' are default, adding more values for gamma
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params = {'C': 10, 'kernel': 'poly'}

In [32]:
# Train the model with the best parameters






best_svm_classifier = SVC(**best_params)
best_svm_classifier.fit(scaled_X_train, y_train)

# Evaluate the model on the test set

y_pred = best_svm_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Parameters: {'C': 10, 'kernel': 'poly'}
Test Accuracy: 0.6825726141078838
Test f1 score: 0.7726597325408618
Test roc auc : 0.6510361784334388
